In [2]:
# Import pakages
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import time

### Making crime data

In [4]:
# Import monthly data from Dec 2023 to Nov 2025
crime_23_25 = pd.read_csv("Data/MPS Borough Level Crime (most recent 24 months).csv")
crime_23_25.head(1)

,MajorText,MinorText,BoroughName,202312,202401,202402,202403,202404,202405,202406,...,202502,202503,202504,202505,202506,202507,202508,202509,202510,202511
0,ARSON AND CRIMINAL DAMAGE,ARSON,Barking and Dagenham,4,4,5,6,3,8,2,...,8,4,12,10,10,11,10,5,6,8


In [5]:
# Remove Unknown, London Heathrow and London City Airports
crime_23_25.BoroughName.unique()

array(['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
       'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney',
       'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering',
       'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea',
       'Kingston upon Thames', 'Lambeth', 'Lewisham',
       'London Heathrow and London City Airports', 'Merton', 'Newham',
       'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton',
       'Tower Hamlets', 'Unknown', 'Waltham Forest', 'Wandsworth',
       'Westminster'], dtype=object)

In [ ]:
# Modify to ensure consistency with data before 2023
crime_23_25[["BoroughName", "MajorText", "MinorText"]].nunique()

BoroughName    34
MajorText      13
MinorText      33
dtype: int64

In [ ]:
# Remove NFIB Fraud to match pre-2023 data 
crime_23_25 = crime_23_25[~(crime_23_25.MajorText == "NFIB FRAUD")]
crime_23_25 = crime_23_25[~(crime_23_25.BoroughName.isin(["Unknown", "London Heathrow and London City Airports" ]))]
crime_23_25[["BoroughName", "MajorText", "MinorText"]].nunique()

BoroughName    32
MajorText      12
MinorText      32
dtype: int64

In [234]:
# checking1 = crime_23_25[['MajorText', 'MinorText']].sort_values(['MajorText', 'MinorText']).drop_duplicates()
# checking1

In [235]:
# Import monthly data from Apr 2010 to Nov 2023
crime_10_22 = pd.read_csv("Data/MPS Borough Level Crime (Historical).csv")
crime_10_22.head(1)

,MajorText,MinorText,BoroughName,201004,201005,201006,201007,201008,201009,201010,...,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311
0,ARSON AND CRIMINAL DAMAGE,ARSON,Barking and Dagenham,6,5,11,10,6,6,13,...,3,2,2,5,2,5,7,4,4,3


In [236]:
crime_10_22 = crime_10_22[~(crime_10_22.BoroughName.isin(["Unknown", "London Heathrow and London City Airports"]))]
crime_10_22[["BoroughName", "MajorText", "MinorText"]].nunique()

BoroughName    32
MajorText      12
MinorText      32
dtype: int64

In [237]:
crime_10_22.BoroughName.unique()

array(['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
       'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney',
       'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering',
       'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea',
       'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham',
       'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton',
       'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster'],
      dtype=object)

In [238]:
# checking2 = crime_10_22[['MajorText', 'MinorText']].sort_values(['MajorText', 'MinorText']).drop_duplicates()
# checking2

- Removed NFIB FRAUD (available only for 2023–2025)
- Before April 2023: burglary recorded as domestic burglary only
- After April 2023: split into multiple residential burglary categories
- All categories merged into Total Residential Burglary for consistency across the dataset

In [ ]:
# 1. Define residential burglary subcategories to be merged
res_burglary_subcats = [
    'BURGLARY - RESIDENTIAL',
    'RES BURGLARY OF A HOME',
    'RES BURGLARY OF UNCONNECTED BUILDING'
]

# 2. Filter rows corresponding to residential burglary subcategories
mask = crime_23_25['MinorText'].isin(res_burglary_subcats)
burglary_df = crime_23_25[mask].copy()


# 3. Aggregate monthly counts by Borough
# Sum all date columns (e.g., 202301, 202302, etc.)
# Standardise category labels
agg_funcs = {col: 'sum' for col in crime_23_25.columns if col.isdigit()} 
agg_funcs['MajorText'] = 'first' # 'BURGLARY' 유지

merged_burglary = burglary_df.groupby('BoroughName').agg(agg_funcs).reset_index()
merged_burglary['MinorText'] = 'TOTAL RESIDENTIAL BURGLARY'
merged_burglary['MajorText'] = 'BURGLARY'

# 4. Remove original subcategories and append the aggregated category
df_cleaned = crime_23_25[~mask].copy()
final_df = pd.concat([df_cleaned, merged_burglary], ignore_index=True)

final_df[final_df['MinorText'] == 'TOTAL RESIDENTIAL BURGLARY'].head()

# 5. Replace dataset with the harmonised version
crime_23_25 = final_df.copy()
crime_23_25.head()

,MajorText,MinorText,BoroughName,202312,202401,202402,202403,202404,202405,202406,...,202502,202503,202504,202505,202506,202507,202508,202509,202510,202511
0,ARSON AND CRIMINAL DAMAGE,ARSON,Barking and Dagenham,4,4,5,6,3,8,2,...,8,4,12,10,10,11,10,5,6,8
1,ARSON AND CRIMINAL DAMAGE,CRIMINAL DAMAGE,Barking and Dagenham,125,126,124,134,112,102,106,...,89,103,124,108,124,137,126,100,100,104
2,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Barking and Dagenham,20,31,21,30,24,28,33,...,14,24,20,27,25,24,23,23,16,21
3,DRUG OFFENCES,POSSESSION OF DRUGS,Barking and Dagenham,45,71,51,64,60,58,64,...,51,57,102,94,80,66,108,79,98,90
4,DRUG OFFENCES,TRAFFICKING OF DRUGS,Barking and Dagenham,19,28,18,42,21,30,31,...,38,36,65,97,46,69,33,37,16,22


In [ ]:
# 1. Define residential burglary categories to be merged
res_burglary_subcats = [
    'DOMESTIC BURGLARY',
    'RES BURGLARY OF A HOME',
    'RES BURGLARY OF UNCONNECTED BUILDING'
]

# 2. Filter rows corresponding to residential burglary categories
mask = crime_10_22['MinorText'].isin(res_burglary_subcats)
burglary_df = crime_10_22[mask].copy()

# 3. Aggregate monthly counts by Borough (sum all date columns)
agg_funcs = {col: 'sum' for col in crime_10_22.columns if col.isdigit()} 
agg_funcs['MajorText'] = 'first'

merged_burglary = burglary_df.groupby('BoroughName').agg(agg_funcs).reset_index()
merged_burglary['MinorText'] = 'TOTAL RESIDENTIAL BURGLARY'
merged_burglary['MajorText'] = 'BURGLARY'

# 4. Remove original categories and append the aggregated category
df_cleaned = crime_10_22[~mask].copy()
final_df = pd.concat([df_cleaned, merged_burglary], ignore_index=True)

# 5. Replace dataset with the harmonised version
final_df[final_df['MinorText'] == 'TOTAL RESIDENTIAL BURGLARY'].head()

crime_10_22 = final_df.copy()
crime_10_22.head()

,MajorText,MinorText,BoroughName,201004,201005,201006,201007,201008,201009,201010,...,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311
0,ARSON AND CRIMINAL DAMAGE,ARSON,Barking and Dagenham,6,5,11,10,6,6,13,...,3,2,2,5,2,5,7,4,4,3
1,ARSON AND CRIMINAL DAMAGE,CRIMINAL DAMAGE,Barking and Dagenham,208,190,218,217,203,161,196,...,91,110,97,108,122,131,114,128,82,88
2,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Barking and Dagenham,49,58,58,46,46,51,53,...,18,32,36,27,28,25,34,39,26,19
3,DRUG OFFENCES,POSSESSION OF DRUGS,Barking and Dagenham,76,64,82,72,98,87,109,...,137,103,71,88,91,129,83,83,58,73
4,DRUG OFFENCES,TRAFFICKING OF DRUGS,Barking and Dagenham,9,6,5,10,9,7,10,...,12,25,24,25,42,22,16,19,25,23


In [241]:
summary = pd.DataFrame({
    '2010-2022' : crime_10_22[["BoroughName", "MajorText", "MinorText"]].nunique(),
    '2023-2025' : crime_23_25[["BoroughName", "MajorText", "MinorText"]].nunique()
})
summary 

,2010-2022,2023-2025
BoroughName,32,32
MajorText,12,12
MinorText,30,30


In [ ]:
# Merge two datasets
crime = crime_10_22.merge(crime_23_25, how = 'right', on = ['MajorText', 'MinorText', 'BoroughName'])
crime.head(1)

,MajorText,MinorText,BoroughName,201004,201005,201006,201007,201008,201009,201010,...,202502,202503,202504,202505,202506,202507,202508,202509,202510,202511
0,ARSON AND CRIMINAL DAMAGE,ARSON,Barking and Dagenham,6,5,11,10,6,6,13,...,8,4,12,10,10,11,10,5,6,8


In [243]:
# Rename the 'MajorText' to 'crime_category', 'MinorText' to 'crime_subcategory'
# 'crime_category' represents broad offence groups, while 'crime_subcategory' provides more detailed classifications within each category.
crime = crime.rename(columns={'MajorText': 'crime_category', 'MinorText': 'crime_subcategory'})
crime.head(3)

,crime_category,crime_subcategory,BoroughName,201004,201005,201006,201007,201008,201009,201010,...,202502,202503,202504,202505,202506,202507,202508,202509,202510,202511
0,ARSON AND CRIMINAL DAMAGE,ARSON,Barking and Dagenham,6,5,11,10,6,6,13,...,8,4,12,10,10,11,10,5,6,8
1,ARSON AND CRIMINAL DAMAGE,CRIMINAL DAMAGE,Barking and Dagenham,208,190,218,217,203,161,196,...,89,103,124,108,124,137,126,100,100,104
2,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Barking and Dagenham,49,58,58,46,46,51,53,...,14,24,20,27,25,24,23,23,16,21


In [244]:
crime.shape

(953, 191)

### Processing crime data

In [ ]:
# Melting the dataframe
id_cols = ['crime_category', 'crime_subcategory', 'BoroughName']

crime_long = crime.melt(
    id_vars=id_cols,
    var_name='year_month',  # The name of the new column that will store the original column names (i.e. the dates)
    value_name='crime_count' # The name of the new column that will store the values from the melted columns.
).assign(year_month = lambda x : x['year_month'].astype(str), 
               year = lambda x : x['year_month'].str[:4].astype(int),
               month = lambda x : x['year_month'].str[4:6].astype(int),
               date = lambda x : pd.to_datetime(x['year_month'], format ='%Y%m'))

crime_long

crime_long.drop(columns=['year_month'], inplace = True)

In [246]:
crime_long.info() # 978 * 188(191-3) = 183864

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179164 entries, 0 to 179163
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   crime_category     179164 non-null  object        
 1   crime_subcategory  179164 non-null  object        
 2   BoroughName        179164 non-null  object        
 3   crime_count        179164 non-null  int64         
 4   year               179164 non-null  int64         
 5   month              179164 non-null  int64         
 6   date               179164 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 9.6+ MB


In [247]:
crime_long[crime_long.crime_count.isna()]

,crime_category,crime_subcategory,BoroughName,crime_count,year,month,date


In [248]:
new_order = ['date', 'year', 'month', 'BoroughName', 'crime_category', 'crime_subcategory', 'crime_count']

crime_long = crime_long[new_order]
crime_long

,date,year,month,BoroughName,crime_category,crime_subcategory,crime_count
0,2010-04-01,2010,4,Barking and Dagenham,ARSON AND CRIMINAL DAMAGE,ARSON,6
1,2010-04-01,2010,4,Barking and Dagenham,ARSON AND CRIMINAL DAMAGE,CRIMINAL DAMAGE,208
2,2010-04-01,2010,4,Barking and Dagenham,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,49
3,2010-04-01,2010,4,Barking and Dagenham,DRUG OFFENCES,POSSESSION OF DRUGS,76
4,2010-04-01,2010,4,Barking and Dagenham,DRUG OFFENCES,TRAFFICKING OF DRUGS,9
...,...,...,...,...,...,...,...
179159,2025-11-01,2025,11,Sutton,BURGLARY,TOTAL RESIDENTIAL BURGLARY,54
179160,2025-11-01,2025,11,Tower Hamlets,BURGLARY,TOTAL RESIDENTIAL BURGLARY,166
179161,2025-11-01,2025,11,Waltham Forest,BURGLARY,TOTAL RESIDENTIAL BURGLARY,83
179162,2025-11-01,2025,11,Wandsworth,BURGLARY,TOTAL RESIDENTIAL BURGLARY,90


In [249]:
crime_long.groupby(['year','month']).sum('crime_count')

crime_count
year month             
2010 4            66453
     5            68540
     6            69238
     7            70500
     8            65275
...                 ...
2025 7            82502
     8            77349
     9            73888
     10           77028
     11           75245

[188 rows x 1 columns]

In [250]:
# Save crime_long data 
# crime_long.to_csv("crime_long.csv", index=False)

### Merge with crime data and population

In [251]:
population = pd.read_excel("Data/population 1801 to 2021.xlsx")
population = population.iloc[0:33, [0, 22, 23]]
population.head(5)

,area,2011,2021
0,City of London,7375.0,8583.0
1,Barking and Dagenham,185911.0,218869.0
2,Barnet,356386.0,389344.0
3,Bexley,231997.0,246472.0
4,Brent,311215.0,339816.0


In [252]:
years_2010 = [str(i) for i in range(2010, 2020)]
years_2020 = [str(y) for y in range(2020, 2025)]
years_2010, years_2020

(['2010',
  '2011',
  '2012',
  '2013',
  '2014',
  '2015',
  '2016',
  '2017',
  '2018',
  '2019'],
 ['2020', '2021', '2022', '2023', '2024'])

In [253]:
for i in years_2010 :
    population[i] = population['2011'].values

for y in years_2020 :
    population[y] = population['2021'].values

population.head()

,area,2011,2021,2010,2012,2013,2014,2015,2016,2017,2018,2019,2020,2022,2023,2024
0,City of London,7375.0,8583.0,7375.0,7375.0,7375.0,7375.0,7375.0,7375.0,7375.0,7375.0,7375.0,8583.0,8583.0,8583.0,8583.0
1,Barking and Dagenham,185911.0,218869.0,185911.0,185911.0,185911.0,185911.0,185911.0,185911.0,185911.0,185911.0,185911.0,218869.0,218869.0,218869.0,218869.0
2,Barnet,356386.0,389344.0,356386.0,356386.0,356386.0,356386.0,356386.0,356386.0,356386.0,356386.0,356386.0,389344.0,389344.0,389344.0,389344.0
3,Bexley,231997.0,246472.0,231997.0,231997.0,231997.0,231997.0,231997.0,231997.0,231997.0,231997.0,231997.0,246472.0,246472.0,246472.0,246472.0
4,Brent,311215.0,339816.0,311215.0,311215.0,311215.0,311215.0,311215.0,311215.0,311215.0,311215.0,311215.0,339816.0,339816.0,339816.0,339816.0


In [254]:
years_all = ['area'] + [str(y) for y in range(2010, 2025)]
years_all

population = population[years_all]
population.head(1)

,area,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,City of London,7375.0,7375.0,7375.0,7375.0,7375.0,7375.0,7375.0,7375.0,7375.0,7375.0,8583.0,8583.0,8583.0,8583.0,8583.0


In [255]:
cols = ['area']
population = population.melt(
    id_vars = cols,
    var_name = 'year',
    value_name = 'population')
population

population.year = population.year.astype(int)
population

,area,year,population
0,City of London,2010,7375.0
1,Barking and Dagenham,2010,185911.0
2,Barnet,2010,356386.0
3,Bexley,2010,231997.0
4,Brent,2010,311215.0
...,...,...,...
490,Sutton,2024,209639.0
491,Tower Hamlets,2024,310306.0
492,Waltham Forest,2024,278425.0
493,Wandsworth,2024,327506.0


In [256]:
crime_long

,date,year,month,BoroughName,crime_category,crime_subcategory,crime_count
0,2010-04-01,2010,4,Barking and Dagenham,ARSON AND CRIMINAL DAMAGE,ARSON,6
1,2010-04-01,2010,4,Barking and Dagenham,ARSON AND CRIMINAL DAMAGE,CRIMINAL DAMAGE,208
2,2010-04-01,2010,4,Barking and Dagenham,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,49
3,2010-04-01,2010,4,Barking and Dagenham,DRUG OFFENCES,POSSESSION OF DRUGS,76
4,2010-04-01,2010,4,Barking and Dagenham,DRUG OFFENCES,TRAFFICKING OF DRUGS,9
...,...,...,...,...,...,...,...
179159,2025-11-01,2025,11,Sutton,BURGLARY,TOTAL RESIDENTIAL BURGLARY,54
179160,2025-11-01,2025,11,Tower Hamlets,BURGLARY,TOTAL RESIDENTIAL BURGLARY,166
179161,2025-11-01,2025,11,Waltham Forest,BURGLARY,TOTAL RESIDENTIAL BURGLARY,83
179162,2025-11-01,2025,11,Wandsworth,BURGLARY,TOTAL RESIDENTIAL BURGLARY,90


In [257]:
# data for crime_population clustering
crime_population = pd.merge(crime_long , population, how = 'left', left_on = ['BoroughName', 'year'], right_on = ['area', 'year'])
crime_population.drop(columns = ['area'], inplace=True)
crime_population

,date,year,month,BoroughName,crime_category,crime_subcategory,crime_count,population
0,2010-04-01,2010,4,Barking and Dagenham,ARSON AND CRIMINAL DAMAGE,ARSON,6,185911.0
1,2010-04-01,2010,4,Barking and Dagenham,ARSON AND CRIMINAL DAMAGE,CRIMINAL DAMAGE,208,185911.0
2,2010-04-01,2010,4,Barking and Dagenham,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,49,185911.0
3,2010-04-01,2010,4,Barking and Dagenham,DRUG OFFENCES,POSSESSION OF DRUGS,76,185911.0
4,2010-04-01,2010,4,Barking and Dagenham,DRUG OFFENCES,TRAFFICKING OF DRUGS,9,185911.0
...,...,...,...,...,...,...,...,...
179159,2025-11-01,2025,11,Sutton,BURGLARY,TOTAL RESIDENTIAL BURGLARY,54,NaN
179160,2025-11-01,2025,11,Tower Hamlets,BURGLARY,TOTAL RESIDENTIAL BURGLARY,166,NaN
179161,2025-11-01,2025,11,Waltham Forest,BURGLARY,TOTAL RESIDENTIAL BURGLARY,83,NaN
179162,2025-11-01,2025,11,Wandsworth,BURGLARY,TOTAL RESIDENTIAL BURGLARY,90,NaN


---

In [258]:
inflation = pd.read_csv("Data/Inflation.csv")
inflation.head(2)

,index date,name,CPIH ALL ITEMS,FOOD AND NON-ALCOHOLIC BEVERAGES,ALCOHOLIC BEVERAGES AND TOBACCO,CLOTHING AND FOOTWEAR,"HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS","FURNITURE, HOUSEHOLD EQUIPMENT AND MAINTENANCE",HEALTH,TRANSPORT,COMMUNICATION,RECREATION & CULTURE,EDUCATION,RESTAURANTS AND HOTELS,MISCELLANEOUS GOODS AND SERVICES
0,198801,Jan-88,46.851,47.901,27.262,191.973,38.714,68.285,37.606,40.338,96.974,73.343,12.344,32.871,45.903
1,198802,Feb-88,46.998,48.174,27.365,193.181,38.791,68.649,37.662,40.317,96.963,73.553,12.348,33.025,46.086


In [259]:
inflation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 15 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   index date                                         455 non-null    int64  
 1   name                                               455 non-null    object 
 2   CPIH ALL ITEMS                                     455 non-null    float64
 3   FOOD AND NON-ALCOHOLIC BEVERAGES                   455 non-null    float64
 4   ALCOHOLIC BEVERAGES AND TOBACCO                    455 non-null    float64
 5   CLOTHING AND FOOTWEAR                              455 non-null    float64
 6   HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS  455 non-null    float64
 7   FURNITURE, HOUSEHOLD EQUIPMENT AND MAINTENANCE     455 non-null    float64
 8   HEALTH                                             455 non-null    float64
 9   TRANSPORT 

In [ ]:
inflation['year'] = (inflation['index date   '] // 100).astype(int)

inflation['month'] = (inflation['index date   '] % 100).astype(int)

inflation['year_month'] = pd.to_datetime(
    inflation['year'].astype(str) + inflation['month'].astype(str).str.zfill(2), 
    format='%Y%m'
)

inflation

inflation.drop(columns = ['index date   ', 'name'], inplace = True)

In [ ]:
cols = ['year_month', 'year', 'month', 'CPIH ALL ITEMS', 'FOOD AND NON-ALCOHOLIC BEVERAGES',
       'ALCOHOLIC BEVERAGES AND TOBACCO', 'CLOTHING AND FOOTWEAR',
       'HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS',
       'FURNITURE, HOUSEHOLD EQUIPMENT AND MAINTENANCE', 'HEALTH', 'TRANSPORT',
       'COMMUNICATION', 'RECREATION & CULTURE', 'EDUCATION',
       'RESTAURANTS AND HOTELS', 'MISCELLANEOUS GOODS AND SERVICES']

inflation = inflation[cols]
inflation 

,year_month,year,month,CPIH ALL ITEMS,FOOD AND NON-ALCOHOLIC BEVERAGES,ALCOHOLIC BEVERAGES AND TOBACCO,CLOTHING AND FOOTWEAR,"HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS","FURNITURE, HOUSEHOLD EQUIPMENT AND MAINTENANCE",HEALTH,TRANSPORT,COMMUNICATION,RECREATION & CULTURE,EDUCATION,RESTAURANTS AND HOTELS,MISCELLANEOUS GOODS AND SERVICES
0,1988-01-01,1988,1,46.851,47.901,27.262,191.973,38.714,68.285,37.606,40.338,96.974,73.343,12.344,32.871,45.903
1,1988-02-01,1988,2,46.998,48.174,27.365,193.181,38.791,68.649,37.662,40.317,96.963,73.553,12.348,33.025,46.086
2,1988-03-01,1988,3,47.202,48.324,27.433,194.760,38.993,69.179,38.192,40.559,96.968,73.580,12.349,33.115,46.108
3,1988-04-01,1988,4,47.801,48.592,27.824,195.128,39.763,69.502,38.385,41.075,96.966,74.924,12.360,33.526,46.369
4,1988-05-01,1988,5,48.044,48.714,27.931,197.851,40.011,69.834,38.513,41.217,96.958,75.220,12.365,33.670,46.538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,2025-07-01,2025,7,138.468,142.584,155.946,117.158,140.222,125.278,139.162,141.089,142.599,134.718,145.336,151.044,123.227
451,2025-08-01,2025,8,138.867,143.160,156.346,118.281,140.587,126.217,139.781,141.703,142.516,135.092,145.336,150.671,123.614
452,2025-09-01,2025,9,138.936,142.876,156.330,121.065,141.032,126.179,140.112,138.445,142.085,135.117,147.451,151.802,123.545
453,2025-10-01,2025,10,139.454,143.604,156.769,122.248,141.668,126.697,139.892,138.500,141.841,135.281,152.704,151.775,123.817


In [262]:
yr_mth_Bor_pop = crime_population[['year', 'month', 'BoroughName', 'population']]
yr_mth_Bor_pop = yr_mth_Bor_pop.drop_duplicates()

In [263]:
# Merge crime_long with inflation
merged_df = yr_mth_Bor_pop.merge(inflation.iloc[:, 1:], how = 'left', on = ['year', 'month'])
merged_df

,year,month,BoroughName,population,CPIH ALL ITEMS,FOOD AND NON-ALCOHOLIC BEVERAGES,ALCOHOLIC BEVERAGES AND TOBACCO,CLOTHING AND FOOTWEAR,"HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS","FURNITURE, HOUSEHOLD EQUIPMENT AND MAINTENANCE",HEALTH,TRANSPORT,COMMUNICATION,RECREATION & CULTURE,EDUCATION,RESTAURANTS AND HOTELS,MISCELLANEOUS GOODS AND SERVICES
0,2010,4,Barking and Dagenham,185911.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064
1,2010,4,Barnet,356386.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064
2,2010,4,Bexley,231997.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064
3,2010,4,Brent,311215.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064
4,2010,4,Bromley,309392.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6011,2025,11,Sutton,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742
6012,2025,11,Tower Hamlets,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742
6013,2025,11,Waltham Forest,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742
6014,2025,11,Wandsworth,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742


In [265]:
merged_df.info() # null (x)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6016 entries, 0 to 6015
Data columns (total 17 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   year                                               6016 non-null   int64  
 1   month                                              6016 non-null   int64  
 2   BoroughName                                        6016 non-null   object 
 3   population                                         5664 non-null   float64
 4   CPIH ALL ITEMS                                     6016 non-null   float64
 5   FOOD AND NON-ALCOHOLIC BEVERAGES                   6016 non-null   float64
 6   ALCOHOLIC BEVERAGES AND TOBACCO                    6016 non-null   float64
 7   CLOTHING AND FOOTWEAR                              6016 non-null   float64
 8   HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS  6016 non-null   float64
 9   FURNITUR

-----

In [266]:
income = pd.read_csv("Data/income.csv")
income = income[['Code', 'Area', '2002', '2003','2004','2005','2006','2007','2008','2009','2010','2011',\
        '2012', '2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024']]
income = income.iloc[2:35]
income.head(5)

,Code,Area,2002,2003,2004,2005,2006,2007,2008,2009,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
2,00AA,City of London,!,!,#,#,#,#,#,762.4,...,#,#,#,909.4,#,#,#,#,#,#
3,00AB,Barking and Dagenham,383.3,354.9,360.1,375.1,417.6,446.7,449.9,448.8,...,436.6,462.2,461.0,479.1,471.0,486.7,508.7,536.6,558.4,612.9
4,00AC,Barnet,427.4,450.1,453.3,442.3,466.1,460.0,502.2,528.1,...,491.0,485.6,522.6,536.6,536.4,538.8,574.9,624.3,633.1,714.6
5,00AD,Bexley,405.0,431.9,438.1,452.4,443.4,469.8,483.9,486.3,...,491.1,487.0,513.0,512.7,553.4,566.5,586.6,634.4,680.2,694.4
6,00AE,Brent,368.4,359.8,395.5,394.7,398.7,421.6,437.0,439.1,...,457.6,467.0,471.0,479.9,524.0,531.8,555.7,556.6,634.6,646.2


In [267]:
23* 33

759

In [268]:
cols = ['Code', 'Area']
income = income.melt(
    id_vars = cols,
    var_name = 'year',
    value_name = 'earnings')
income # 22 * 33 (2002 ~ 2024)

,Code,Area,year,earnings
0,00AA,City of London,2002,!
1,00AB,Barking and Dagenham,2002,383.3
2,00AC,Barnet,2002,427.4
3,00AD,Bexley,2002,405.0
4,00AE,Brent,2002,368.4
...,...,...,...,...
754,00BF,Sutton,2024,728.7
755,00BG,Tower Hamlets,2024,766.5
756,00BH,Waltham Forest,2024,717.7
757,00BJ,Wandsworth,2024,843.3


In [269]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759 entries, 0 to 758
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Code      759 non-null    object
 1   Area      759 non-null    object
 2   year      759 non-null    object
 3   earnings  759 non-null    object
dtypes: object(4)
memory usage: 23.8+ KB


In [270]:
income['year'] = income['year'].astype(int)

In [271]:
merged_df

,year,month,BoroughName,population,CPIH ALL ITEMS,FOOD AND NON-ALCOHOLIC BEVERAGES,ALCOHOLIC BEVERAGES AND TOBACCO,CLOTHING AND FOOTWEAR,"HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS","FURNITURE, HOUSEHOLD EQUIPMENT AND MAINTENANCE",HEALTH,TRANSPORT,COMMUNICATION,RECREATION & CULTURE,EDUCATION,RESTAURANTS AND HOTELS,MISCELLANEOUS GOODS AND SERVICES
0,2010,4,Barking and Dagenham,185911.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064
1,2010,4,Barnet,356386.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064
2,2010,4,Bexley,231997.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064
3,2010,4,Brent,311215.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064
4,2010,4,Bromley,309392.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6011,2025,11,Sutton,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742
6012,2025,11,Tower Hamlets,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742
6013,2025,11,Waltham Forest,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742
6014,2025,11,Wandsworth,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742


In [272]:
# crime + inflation + income
merged_df1 = pd.merge(merged_df, income.iloc[:, 1:], how = 'left', left_on = ['year', 'BoroughName'], right_on = ['year', 'Area'])
merged_df1.drop(columns=['Area'], inplace = True)
merged_df1.info() #183864 rows, earings 2025 null values

# city of London will drop 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6016 entries, 0 to 6015
Data columns (total 18 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   year                                               6016 non-null   int64  
 1   month                                              6016 non-null   int64  
 2   BoroughName                                        6016 non-null   object 
 3   population                                         5664 non-null   float64
 4   CPIH ALL ITEMS                                     6016 non-null   float64
 5   FOOD AND NON-ALCOHOLIC BEVERAGES                   6016 non-null   float64
 6   ALCOHOLIC BEVERAGES AND TOBACCO                    6016 non-null   float64
 7   CLOTHING AND FOOTWEAR                              6016 non-null   float64
 8   HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS  6016 non-null   float64
 9   FURNITUR

In [273]:
merged_df1

,year,month,BoroughName,population,CPIH ALL ITEMS,FOOD AND NON-ALCOHOLIC BEVERAGES,ALCOHOLIC BEVERAGES AND TOBACCO,CLOTHING AND FOOTWEAR,"HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS","FURNITURE, HOUSEHOLD EQUIPMENT AND MAINTENANCE",HEALTH,TRANSPORT,COMMUNICATION,RECREATION & CULTURE,EDUCATION,RESTAURANTS AND HOTELS,MISCELLANEOUS GOODS AND SERVICES,earnings
0,2010,4,Barking and Dagenham,185911.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064,429.3
1,2010,4,Barnet,356386.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064,501.6
2,2010,4,Bexley,231997.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064,518.1
3,2010,4,Brent,311215.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064,423.1
4,2010,4,Bromley,309392.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064,527.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6011,2025,11,Sutton,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742,NaN
6012,2025,11,Tower Hamlets,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742,NaN
6013,2025,11,Waltham Forest,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742,NaN
6014,2025,11,Wandsworth,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742,NaN


In [274]:
# 2010년 4월의 33개의 구에서 물가는 같은데 여기에 지역보정이 들어가면 다 다른값이 들어갈 예정
# 2011년 5월의 33개의 구에서 물가는 같은데 지역보정 들어가면 다 다른값 들어갈 것. 단 2010년 4월과 2010년 5월은 값이 달라서 다 다르게 들어갈거 같은데..?dd
# merged_df1.drop(columns = ['Code', 'Area'], inplace = True)
merged_df1.earnings = merged_df1.earnings.astype(float)

----

In [275]:
business_demo = pd.read_csv("Data/business-demographics.csv")
business_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0                       969 non-null    object 
 1   area                969 non-null    object 
 2   year                969 non-null    int64  
 3   active_enterprises  969 non-null    int64  
 4   births              969 non-null    int64  
 5   birth_rate          969 non-null    float64
 6   deaths              969 non-null    int64  
 7   death_rate          969 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 60.7+ KB


In [276]:
business_demo = business_demo.iloc[:, 1:]
business_demo # 2004 ~ 2022

,area,year,active_enterprises,births,birth_rate,deaths,death_rate
0,City of London,2004,11980,1175,9.8,1160,9.7
1,Barking and Dagenham,2004,3120,480,15.4,410,13.1
2,Barnet,2004,19655,2435,12.4,3015,15.3
3,Bexley,2004,6450,940,14.6,800,12.4
4,Brent,2004,11295,1800,15.9,1745,15.4
...,...,...,...,...,...,...,...
964,Scotland,2022,174340,18870,10.8,19640,11.3
965,Northern Ireland,2022,64745,5385,8.3,5290,8.2
966,England And Wales,2022,2685600,312670,11.6,320560,11.9
967,Great Britain,2022,2859940,331540,11.6,340200,11.9


In [277]:
business_demo.area.nunique() # England

51

In [278]:
merged_df2 = merged_df1.merge(business_demo[['area', 'year', 'birth_rate', 'death_rate']], how = 'left', left_on = ['BoroughName', 'year'], right_on = ['area', 'year'])
merged_df2.drop(columns=['area'], inplace = True)
merged_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6016 entries, 0 to 6015
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   year                                               6016 non-null   int64  
 1   month                                              6016 non-null   int64  
 2   BoroughName                                        6016 non-null   object 
 3   population                                         5664 non-null   float64
 4   CPIH ALL ITEMS                                     6016 non-null   float64
 5   FOOD AND NON-ALCOHOLIC BEVERAGES                   6016 non-null   float64
 6   ALCOHOLIC BEVERAGES AND TOBACCO                    6016 non-null   float64
 7   CLOTHING AND FOOTWEAR                              6016 non-null   float64
 8   HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS  6016 non-null   float64
 9   FURNITUR

In [279]:
merged_df2.BoroughName.nunique()

32

In [280]:
merged_df2

,year,month,BoroughName,population,CPIH ALL ITEMS,FOOD AND NON-ALCOHOLIC BEVERAGES,ALCOHOLIC BEVERAGES AND TOBACCO,CLOTHING AND FOOTWEAR,"HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS","FURNITURE, HOUSEHOLD EQUIPMENT AND MAINTENANCE",HEALTH,TRANSPORT,COMMUNICATION,RECREATION & CULTURE,EDUCATION,RESTAURANTS AND HOTELS,MISCELLANEOUS GOODS AND SERVICES,earnings,birth_rate,death_rate
0,2010,4,Barking and Dagenham,185911.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064,429.3,15.2,14.0
1,2010,4,Barnet,356386.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064,501.6,11.8,13.5
2,2010,4,Bexley,231997.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064,518.1,11.7,11.8
3,2010,4,Brent,311215.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064,423.1,12.9,11.2
4,2010,4,Bromley,309392.0,89.886,90.362,77.143,96.538,89.365,90.051,86.927,91.206,88.215,98.207,61.479,86.332,94.064,527.2,11.4,10.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6011,2025,11,Sutton,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742,NaN,NaN,NaN
6012,2025,11,Tower Hamlets,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742,NaN,NaN,NaN
6013,2025,11,Waltham Forest,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742,NaN,NaN,NaN
6014,2025,11,Wandsworth,NaN,139.361,143.329,156.129,121.858,142.253,125.762,139.742,137.329,141.447,135.420,152.704,151.474,123.742,NaN,NaN,NaN


---

In [281]:
# Import interest data
interest = pd.read_csv("Data/Bank Rate history and data  Bank of England Database.csv")
interest

interest['Date Changed'] = pd.to_datetime(interest['Date Changed'], format='%d %b %y')
interest = interest[interest['Date Changed'].dt.year >= 2010]
interest


,Date Changed,Rate
0,2025-12-18,3.75
1,2025-08-07,4.00
2,2025-05-08,4.25
3,2025-02-06,4.50
4,2024-11-07,4.75
5,2024-08-01,5.00
6,2023-08-03,5.25
7,2023-06-22,5.00
8,2023-05-11,4.50
9,2023-03-23,4.25


In [ ]:
# Sort interest rate changes by date
interest = interest.sort_values('Date Changed')
# Set date column as index for time-series operations
interest = interest.set_index('Date Changed')

# Resample to monthly frequency (Month Start)
# Use the last rate in each month and forward-fill missing months
monthly_rate = (
    interest['Rate']
    .resample('MS')   # MS = Month Start
    .last()           # Last available rate in the month
    .ffill()          # Carry forward previous rate if no change
    .reset_index()
)

# Extract year and month for merging with other datasets
monthly_rate['year'] = monthly_rate['Date Changed'].dt.year
monthly_rate['month'] = monthly_rate['Date Changed'].dt.month

# Keep only columns required for analysis
monthly_rate = monthly_rate[['year', 'month', 'Rate']]

print(monthly_rate.head(15))
print(monthly_rate.tail(15))

    year  month  Rate
0   2016      8  0.25
1   2016      9  0.25
2   2016     10  0.25
3   2016     11  0.25
4   2016     12  0.25
5   2017      1  0.25
6   2017      2  0.25
7   2017      3  0.25
8   2017      4  0.25
9   2017      5  0.25
10  2017      6  0.25
11  2017      7  0.25
12  2017      8  0.25
13  2017      9  0.25
14  2017     10  0.25
     year  month  Rate
98   2024     10  5.00
99   2024     11  4.75
100  2024     12  4.75
101  2025      1  4.75
102  2025      2  4.50
103  2025      3  4.50
104  2025      4  4.50
105  2025      5  4.25
106  2025      6  4.25
107  2025      7  4.25
108  2025      8  4.00
109  2025      9  4.00
110  2025     10  4.00
111  2025     11  4.00
112  2025     12  3.75


In [ ]:
# Fill missing historical period (Jan 2010 – Jul 2016)
# Interest rate remained constant at 0.5 during this period

# Generate monthly date range for the missing period
missing_period = pd.date_range(
    start="2010-01-01",
    end="2016-07-01",
    freq="MS"  
)

# Create DataFrame with constant interest rate
df_missing = pd.DataFrame({
    "year": missing_period.year,
    "month": missing_period.month,
    "Rate": 0.5
})

# Recreate date column in existing data for alignment
monthly_rate['date'] = pd.to_datetime(
    monthly_rate['year'].astype(str) + "-" +
    monthly_rate['month'].astype(str) + "-01"
)

# Combine historical fixed-rate period with existing monthly data
final_rate = pd.concat(
    [df_missing, monthly_rate[['year', 'month', 'Rate']]],
    ignore_index=True
)

# Remove duplicates (keep observed data where overlap exists)
final_rate = (
    final_rate
    .drop_duplicates(subset=['year', 'month'], keep='last')
    .sort_values(['year', 'month'])
    .reset_index(drop=True)
)

final_rate.head(20)
final_rate = final_rate.rename({'Rate' : 'interest_rate'})

In [284]:
final_rate.tail()

,year,month,Rate
187,2025,8,4.00
188,2025,9,4.00
189,2025,10,4.00
190,2025,11,4.00
191,2025,12,3.75


In [285]:
final_rate.info() # 12 * 16

<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, 0 to 191
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    192 non-null    int32  
 1   month   192 non-null    int32  
 2   Rate    192 non-null    float64
dtypes: float64(1), int32(2)
memory usage: 4.5 KB


In [286]:
merged_df3 = pd.merge(merged_df2 , final_rate, how = 'left', left_on = ['year', 'month'], right_on = ['year', 'month'])
merged_df3

,year,month,BoroughName,population,CPIH ALL ITEMS,FOOD AND NON-ALCOHOLIC BEVERAGES,ALCOHOLIC BEVERAGES AND TOBACCO,CLOTHING AND FOOTWEAR,"HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS","FURNITURE, HOUSEHOLD EQUIPMENT AND MAINTENANCE",...,TRANSPORT,COMMUNICATION,RECREATION & CULTURE,EDUCATION,RESTAURANTS AND HOTELS,MISCELLANEOUS GOODS AND SERVICES,earnings,birth_rate,death_rate,Rate
0,2010,4,Barking and Dagenham,185911.0,89.886,90.362,77.143,96.538,89.365,90.051,...,91.206,88.215,98.207,61.479,86.332,94.064,429.3,15.2,14.0,0.5
1,2010,4,Barnet,356386.0,89.886,90.362,77.143,96.538,89.365,90.051,...,91.206,88.215,98.207,61.479,86.332,94.064,501.6,11.8,13.5,0.5
2,2010,4,Bexley,231997.0,89.886,90.362,77.143,96.538,89.365,90.051,...,91.206,88.215,98.207,61.479,86.332,94.064,518.1,11.7,11.8,0.5
3,2010,4,Brent,311215.0,89.886,90.362,77.143,96.538,89.365,90.051,...,91.206,88.215,98.207,61.479,86.332,94.064,423.1,12.9,11.2,0.5
4,2010,4,Bromley,309392.0,89.886,90.362,77.143,96.538,89.365,90.051,...,91.206,88.215,98.207,61.479,86.332,94.064,527.2,11.4,10.9,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6011,2025,11,Sutton,NaN,139.361,143.329,156.129,121.858,142.253,125.762,...,137.329,141.447,135.420,152.704,151.474,123.742,NaN,NaN,NaN,4.0
6012,2025,11,Tower Hamlets,NaN,139.361,143.329,156.129,121.858,142.253,125.762,...,137.329,141.447,135.420,152.704,151.474,123.742,NaN,NaN,NaN,4.0
6013,2025,11,Waltham Forest,NaN,139.361,143.329,156.129,121.858,142.253,125.762,...,137.329,141.447,135.420,152.704,151.474,123.742,NaN,NaN,NaN,4.0
6014,2025,11,Wandsworth,NaN,139.361,143.329,156.129,121.858,142.253,125.762,...,137.329,141.447,135.420,152.704,151.474,123.742,NaN,NaN,NaN,4.0


In [287]:
# interest rate - no null values
merged_df3 = merged_df3.rename(columns={'Rate' : 'interest_rate'})
merged_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6016 entries, 0 to 6015
Data columns (total 21 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   year                                               6016 non-null   int64  
 1   month                                              6016 non-null   int64  
 2   BoroughName                                        6016 non-null   object 
 3   population                                         5664 non-null   float64
 4   CPIH ALL ITEMS                                     6016 non-null   float64
 5   FOOD AND NON-ALCOHOLIC BEVERAGES                   6016 non-null   float64
 6   ALCOHOLIC BEVERAGES AND TOBACCO                    6016 non-null   float64
 7   CLOTHING AND FOOTWEAR                              6016 non-null   float64
 8   HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS  6016 non-null   float64
 9   FURNITUR

---

In [ ]:
unemployment = pd.read_csv(
    "Data/Unemployment_london(Nomis).csv",
    encoding="latin1",
    sep=",",
    skiprows=5,   # Skip the first five rows (metadata/header information)
    header=1,     # Use the next row as column names
    engine="python"
)

unemployment.head()


,Area,Jan 2004-Dec 2004,Conf,Jan 2005-Dec 2005,Conf.1,Jan 2006-Dec 2006,Conf.2,Jan 2007-Dec 2007,Conf.3,Jan 2008-Dec 2008,...,Jan 2020-Dec 2020,Conf.16,Jan 2021-Dec 2021,Conf.17,Jan 2022-Dec 2022,Conf.18,Jan 2023-Dec 2023,Conf.19,Jan 2024-Dec 2024,Conf.20
0,ladu2023:Barking and Dagenham,8.3,1.8,9.1,1.8,9.5,2.0,7.8,1.7,8.6,...,8.0,2.1,7.6,2.0,4.9,2.0,6.8,2.0,6.4,2.0
1,ladu2023:Barnet,6.3,1.3,7.0,1.4,6.8,1.6,5.2,1.1,6.4,...,5.7,1.4,6.4,1.5,5.3,1.8,5.4,1.7,4.0,1.4
2,ladu2023:Bexley,4.1,1.0,5.0,1.1,5.5,1.4,4.4,1.0,5.6,...,4.9,1.3,5.1,1.2,2.7,1.1,4.9,1.5,3.0,1.1
3,ladu2023:Brent,9.1,1.8,7.9,1.6,9.1,2.1,8.7,1.8,7.3,...,6.5,1.9,6.4,1.9,7.0,2.4,5.7,2.3,6.7,2.4
4,ladu2023:Bromley,4.7,1.1,4.7,1.0,5.1,1.3,4.4,1.0,4.5,...,4.9,1.2,4.7,1.1,2.7,1.1,5.0,1.4,3.7,1.2


In [289]:
first_col = unemployment.iloc[:, [0]]
other_cols = unemployment.iloc[:, 1::2]

unemployment = pd.concat([first_col, other_cols], axis=1)

In [290]:
unemployment = unemployment.iloc[0:33]
unemployment.Area = unemployment.Area.str.split(":").str[1]
unemployment

import re

unemployment.columns = [
    col if col in ['Area', 'BoroughName'] else re.search(r'\d{4}', col).group()
    for col in unemployment.columns
]
unemployment.head(5)

,Area,2004,2005,2006,2007,2008,2009,2010,2011,2012,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Barking and Dagenham,8.3,9.1,9.5,7.8,8.6,11.9,13.1,13.0,13.5,...,9.7,6.8,7.7,5.9,5.7,8.0,7.6,4.9,6.8,6.4
1,Barnet,6.3,7.0,6.8,5.2,6.4,7.5,7.0,8.0,8.9,...,5.8,4.9,4.4,4.7,4.5,5.7,6.4,5.3,5.4,4.0
2,Bexley,4.1,5.0,5.5,4.4,5.6,8.2,8.5,8.0,7.6,...,6.0,4.2,4.6,4.0,4.2,4.9,5.1,2.7,4.9,3.0
3,Brent,9.1,7.9,9.1,8.7,7.3,9.4,8.9,10.8,10.3,...,7.1,7.1,6.3,5.3,4.7,6.5,6.4,7.0,5.7,6.7
4,Bromley,4.7,4.7,5.1,4.4,4.5,6.2,6.9,6.9,6.6,...,4.9,4.1,4.2,4.2,3.0,4.9,4.7,2.7,5.0,3.7


In [291]:
# unemployment.info() # 33 * 22

In [292]:
cols = ['Area']
unemployment = unemployment.melt(
    id_vars = cols,
    var_name = 'year',
    value_name = 'unemployment_rate')
unemployment  # 33 * 21

,Area,year,unemployment_rate
0,Barking and Dagenham,2004,8.3
1,Barnet,2004,6.3
2,Bexley,2004,4.1
3,Brent,2004,9.1
4,Bromley,2004,4.7
...,...,...,...
688,Sutton,2024,3.7
689,Tower Hamlets,2024,5.9
690,Waltham Forest,2024,3.9
691,Wandsworth,2024,4.4


In [293]:
unemployment.year = unemployment.year.astype(int)

In [294]:
merged_df4 = pd.merge(merged_df3 , unemployment, how = 'left', left_on = ['BoroughName', 'year'], right_on = ['Area', 'year'])
merged_df4.drop(columns=['Area'], inplace = True)
merged_df4.head()

,year,month,BoroughName,population,CPIH ALL ITEMS,FOOD AND NON-ALCOHOLIC BEVERAGES,ALCOHOLIC BEVERAGES AND TOBACCO,CLOTHING AND FOOTWEAR,"HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS","FURNITURE, HOUSEHOLD EQUIPMENT AND MAINTENANCE",...,COMMUNICATION,RECREATION & CULTURE,EDUCATION,RESTAURANTS AND HOTELS,MISCELLANEOUS GOODS AND SERVICES,earnings,birth_rate,death_rate,interest_rate,unemployment_rate
0,2010,4,Barking and Dagenham,185911.0,89.886,90.362,77.143,96.538,89.365,90.051,...,88.215,98.207,61.479,86.332,94.064,429.3,15.2,14.0,0.5,13.1
1,2010,4,Barnet,356386.0,89.886,90.362,77.143,96.538,89.365,90.051,...,88.215,98.207,61.479,86.332,94.064,501.6,11.8,13.5,0.5,7.0
2,2010,4,Bexley,231997.0,89.886,90.362,77.143,96.538,89.365,90.051,...,88.215,98.207,61.479,86.332,94.064,518.1,11.7,11.8,0.5,8.5
3,2010,4,Brent,311215.0,89.886,90.362,77.143,96.538,89.365,90.051,...,88.215,98.207,61.479,86.332,94.064,423.1,12.9,11.2,0.5,8.9
4,2010,4,Bromley,309392.0,89.886,90.362,77.143,96.538,89.365,90.051,...,88.215,98.207,61.479,86.332,94.064,527.2,11.4,10.9,0.5,6.9


In [ ]:
merged_df4.info()
# region_econ

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6016 entries, 0 to 6015
Data columns (total 22 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   year                                               6016 non-null   int64  
 1   month                                              6016 non-null   int64  
 2   BoroughName                                        6016 non-null   object 
 3   population                                         5664 non-null   float64
 4   CPIH ALL ITEMS                                     6016 non-null   float64
 5   FOOD AND NON-ALCOHOLIC BEVERAGES                   6016 non-null   float64
 6   ALCOHOLIC BEVERAGES AND TOBACCO                    6016 non-null   float64
 7   CLOTHING AND FOOTWEAR                              6016 non-null   float64
 8   HOUSING, WATER, ELELCTRICITY, GAS AND OTHER FUELS  6016 non-null   float64
 9   FURNITUR

----